# L03: Function Calling Comparison

**Week 3 - Tool Use and Function Calling**

## Learning Objectives
- Compare OpenAI and Anthropic function calling APIs
- Implement tool definitions for both providers
- Handle tool call responses from each API
- Understand the complete tool use loop

## 1. Setup

This notebook demonstrates the API structures. For production use, install the official SDKs.

In [ ]:
import json
from typing import Any

# API keys would be loaded from environment in production
# import os
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

print("Setup complete")

## 2. Tool Definition Comparison

Both APIs use JSON Schema for parameters, but with different structures.

In [ ]:
# OpenAI Tool Definition Format
openai_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current weather for a location. Returns temperature and conditions.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City name, e.g., 'New York' or 'London'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    }
}

print("OpenAI Tool Format:")
print(json.dumps(openai_tool, indent=2))

In [ ]:
# Anthropic Tool Definition Format
anthropic_tool = {
    "name": "get_weather",
    "description": "Get the current weather for a location. Returns temperature and conditions.",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name, e.g., 'New York' or 'London'"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature unit"
            }
        },
        "required": ["location"]
    }
}

print("Anthropic Tool Format:")
print(json.dumps(anthropic_tool, indent=2))

## 3. Key Differences

| Aspect | OpenAI | Anthropic |
|--------|--------|----------|
| Wrapper | `{"type": "function", "function": {...}}` | Direct object |
| Parameters key | `parameters` | `input_schema` |
| Response type | `tool_calls` array | `tool_use` content block |
| Result format | `tool` role message | `tool_result` content block |

In [ ]:
def convert_anthropic_to_openai(anthropic_tool: dict) -> dict:
    """Convert Anthropic tool format to OpenAI format."""
    return {
        "type": "function",
        "function": {
            "name": anthropic_tool["name"],
            "description": anthropic_tool["description"],
            "parameters": anthropic_tool["input_schema"]
        }
    }

def convert_openai_to_anthropic(openai_tool: dict) -> dict:
    """Convert OpenAI tool format to Anthropic format."""
    func = openai_tool["function"]
    return {
        "name": func["name"],
        "description": func["description"],
        "input_schema": func["parameters"]
    }

# Test conversion
converted = convert_openai_to_anthropic(openai_tool)
print("OpenAI -> Anthropic:")
print(json.dumps(converted, indent=2))

## 4. Simulated API Responses

In [ ]:
# OpenAI response when model wants to call a tool
openai_tool_call_response = {
    "id": "chatcmpl-abc123",
    "object": "chat.completion",
    "model": "gpt-4",
    "choices": [{
        "index": 0,
        "message": {
            "role": "assistant",
            "content": None,
            "tool_calls": [{
                "id": "call_xyz789",
                "type": "function",
                "function": {
                    "name": "get_weather",
                    "arguments": '{"location": "New York", "unit": "fahrenheit"}'
                }
            }]
        },
        "finish_reason": "tool_calls"
    }]
}

print("OpenAI Tool Call Response:")
print(json.dumps(openai_tool_call_response, indent=2))

In [ ]:
# Anthropic response when model wants to call a tool
anthropic_tool_use_response = {
    "id": "msg_abc123",
    "type": "message",
    "model": "claude-3-opus-20240229",
    "content": [
        {
            "type": "text",
            "text": "I'll check the weather in New York for you."
        },
        {
            "type": "tool_use",
            "id": "toolu_xyz789",
            "name": "get_weather",
            "input": {
                "location": "New York",
                "unit": "fahrenheit"
            }
        }
    ],
    "stop_reason": "tool_use"
}

print("Anthropic Tool Use Response:")
print(json.dumps(anthropic_tool_use_response, indent=2))

## 5. Processing Tool Calls

In [ ]:
def process_openai_tool_calls(response: dict) -> list:
    """Extract tool calls from OpenAI response."""
    tool_calls = []
    message = response["choices"][0]["message"]
    
    if "tool_calls" in message and message["tool_calls"]:
        for tc in message["tool_calls"]:
            tool_calls.append({
                "id": tc["id"],
                "name": tc["function"]["name"],
                "arguments": json.loads(tc["function"]["arguments"])
            })
    
    return tool_calls

def process_anthropic_tool_use(response: dict) -> list:
    """Extract tool uses from Anthropic response."""
    tool_uses = []
    
    for block in response["content"]:
        if block["type"] == "tool_use":
            tool_uses.append({
                "id": block["id"],
                "name": block["name"],
                "arguments": block["input"]
            })
    
    return tool_uses

# Test processing
print("OpenAI extracted:", process_openai_tool_calls(openai_tool_call_response))
print("Anthropic extracted:", process_anthropic_tool_use(anthropic_tool_use_response))

## 6. Sending Tool Results Back

In [ ]:
# OpenAI: Tool result as a message with role "tool"
openai_tool_result_message = {
    "role": "tool",
    "tool_call_id": "call_xyz789",
    "content": '{"temperature": 45, "unit": "fahrenheit", "condition": "cloudy"}'
}

print("OpenAI Tool Result Message:")
print(json.dumps(openai_tool_result_message, indent=2))

In [ ]:
# Anthropic: Tool result as a content block
anthropic_tool_result_message = {
    "role": "user",
    "content": [{
        "type": "tool_result",
        "tool_use_id": "toolu_xyz789",
        "content": '{"temperature": 45, "unit": "fahrenheit", "condition": "cloudy"}'
    }]
}

print("Anthropic Tool Result Message:")
print(json.dumps(anthropic_tool_result_message, indent=2))

## 7. Tool Choice Options

Both APIs allow controlling when tools are used.

In [ ]:
# OpenAI tool_choice options
openai_tool_choices = {
    "auto": "auto",  # Model decides (default)
    "none": "none",  # Never use tools
    "required": "required",  # Must use at least one tool
    "specific": {"type": "function", "function": {"name": "get_weather"}}  # Force specific tool
}

# Anthropic tool_choice options
anthropic_tool_choices = {
    "auto": {"type": "auto"},  # Model decides (default)
    "any": {"type": "any"},  # Must use at least one tool
    "specific": {"type": "tool", "name": "get_weather"}  # Force specific tool
}

print("OpenAI tool_choice options:")
for name, choice in openai_tool_choices.items():
    print(f"  {name}: {choice}")

print("\nAnthropic tool_choice options:")
for name, choice in anthropic_tool_choices.items():
    print(f"  {name}: {choice}")

## 8. Complete Tool Use Loop (Simulated)

In [ ]:
def simulate_tool_execution(tool_name: str, arguments: dict) -> str:
    """Simulated tool execution."""
    if tool_name == "get_weather":
        return json.dumps({
            "temperature": 45,
            "unit": arguments.get("unit", "fahrenheit"),
            "condition": "cloudy",
            "location": arguments["location"]
        })
    return json.dumps({"error": "Unknown tool"})

def complete_tool_loop(provider: str, response: dict):
    """Demonstrate complete tool use loop."""
    print(f"\n{'='*60}")
    print(f"Provider: {provider}")
    print("="*60)
    
    # Step 1: Extract tool calls
    if provider == "openai":
        tool_calls = process_openai_tool_calls(response)
    else:
        tool_calls = process_anthropic_tool_use(response)
    
    print(f"\nStep 1 - Extracted tool calls: {tool_calls}")
    
    # Step 2: Execute each tool
    results = []
    for tc in tool_calls:
        result = simulate_tool_execution(tc["name"], tc["arguments"])
        results.append({"id": tc["id"], "result": result})
        print(f"\nStep 2 - Executed {tc['name']}: {result}")
    
    # Step 3: Format results for API
    if provider == "openai":
        formatted = [{"role": "tool", "tool_call_id": r["id"], "content": r["result"]} for r in results]
    else:
        formatted = {"role": "user", "content": [{"type": "tool_result", "tool_use_id": r["id"], "content": r["result"]} for r in results]}
    
    print(f"\nStep 3 - Formatted for API:")
    print(json.dumps(formatted, indent=2))
    
    print(f"\nStep 4 - Would send formatted results back to {provider} API for final response")

# Run for both providers
complete_tool_loop("openai", openai_tool_call_response)
complete_tool_loop("anthropic", anthropic_tool_use_response)

## 9. Best Practices

1. **Clear Descriptions**: Detailed tool descriptions help the model choose correctly
2. **Parameter Validation**: Always validate tool inputs before execution
3. **Error Handling**: Return structured error messages for tool failures
4. **Limit Tool Count**: 3-5 tools per request for best selection accuracy
5. **Consistent Naming**: Use verb_noun format (e.g., `get_weather`, `search_web`)

## Key Takeaways

- OpenAI wraps tools in `{"type": "function", "function": {...}}`
- Anthropic uses flat structure with `input_schema`
- Both return structured tool calls that you execute
- Tool results are sent back differently (role vs content block)
- The underlying concepts are the same across providers